In [4]:
!pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/e0/63/b428aaca15fcd98c39b07ca7149e24bc14205ad0f1c80ba2b01835aedde1/pip-23.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.5 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1


In [3]:
!pip install datasets --quiet

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
distributed 2022.7.0 requires tornado<6.2,>=6.0.3, but you have tornado 6.3.3 which is incompatible.

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: pip install --upgrade pip


In [16]:
!pip install requests --quiet

In [8]:
from datasets import load_dataset
import pandas as pd

In [17]:
import os
import requests
import boto3

In [5]:
dataset = load_dataset("phosseini/multimodal_satire")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [9]:
images_df = pd.DataFrame(dataset['train'])

In [14]:
images_df.head(5)

,url,headline,image_link,is_satire
0,https://babylonbee.com/news/this-aint-it-chief...,"'This Ain’t It, Chief' Updated To 'This Ain’t ...",https://babylonbee.com/img/articles/article-65...,1
1,https://babylonbee.com/news/announcement-the-b...,Using Our Prophetic Powers For Good: The Babyl...,https://babylonbee.com/img/articles/article-65...,1
2,https://babylonbee.com/news/study-wearing-a-ma...,Study Finds Connection Between Wearing A Mask ...,https://babylonbee.com/img/articles/article-65...,1
3,https://babylonbee.com/news/benny-hinn-flies-p...,Benny Hinn Travels To Dagobah To Master His Po...,https://babylonbee.com/img/articles/article-65...,1
4,https://babylonbee.com/news/white-house-adds-c...,White House Adds Cry Room For Fussy Reporters,https://babylonbee.com/img/articles/article-65...,1


In [15]:
images_df.head(1).image_link

0    https://babylonbee.com/img/articles/article-65...
Name: image_link, dtype: object

In [26]:
## DO NOT RUN THIS CELL MORE THAN ONCE ##

import io

# Initialize S3 client
s3 = boto3.client('s3')

# S3 bucket and prefix where you want to store the images
s3_bucket = "satire-spotter-data"
s3_prefix = "images/"

# Iterate through the DataFrame to download and upload images
for index, row in images_df.iterrows():
    image_url = row['image_link']
    is_satire = row['is_satire']
    
    # Skip processing if the image_url contains 'babylonbee.com'
    if 'babylonbee.com' in image_url:
        print(f"Skipped: {image_url} (Contains 'babylonbee.com')")
        continue
    
    try:
        # Send an HTTP GET request to the image URL
        response = requests.get(image_url)
        
        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Extract the file extension from the URL
            file_extension = os.path.splitext(image_url)[-1].lower()
            
            # Determine the subfolder based on is_satire
            subfolder = "satire" if is_satire == 1 else "true"
            
            # Generate a unique S3 object key within the subfolder
            s3_object_key = f"{s3_prefix}{subfolder}/image_{index}{file_extension}"
            
            # Convert response.content to a file-like object using io.BytesIO
            image_fileobj = io.BytesIO(response.content)
            
            # Upload the image to S3
            s3.upload_fileobj(image_fileobj, s3_bucket, s3_object_key)
            
            print(f"Uploaded: {image_url} -> s3://{s3_bucket}/{s3_object_key}")
        else:
            print(f"Failed to download: {image_url} (Status Code: {response.status_code})")
    except Exception as e:
        print(f"Error downloading {image_url}: {str(e)}")

Skipped: https://babylonbee.com/img/articles/article-6580-1.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6568-2.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6569-2.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6578-2.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6577-2.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6576-1.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6575-2.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6574-1.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6572-1.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6562-1.jpg (Contains 'babylonbee.com')
Skipped: https://babylonbee.com/img/articles/article-6573-1.jpg (Contains 'babyl